# 1-2-3 of block matrix encoding

## 0 - we need a matrix :)

Let it be some clear matrix, e.g. ceil to the closest even number. We will also limit the number of elements to 7.

In [ ]:
import numpy as np
import math

inc = np.zeros((7, 7), dtype=np.complex128)

for i in range(7):
    inc[math.ceil(i/2) * 2 % 8, i] = 1.0

print(inc)

In [ ]:
# is it unitary? Noooo...
(inc @ inc.T.conj()).real

## 1 - pad it!

I'd say this is unnecessary step for modelling a unitary matrix, but we are only allowed to operate $2^n$-dimensional statevectors.

In [ ]:
A = np.zeros((
             2 ** math.ceil(
                    math.log2(inc.shape[0])   # round up to pthe power of 2.
             ),
             2 ** math.ceil(
                    math.log2(inc.shape[1])
             )
             )
    , dtype=np.complex128
)
A[:inc.shape[0], :inc.shape[1]] = inc

import matplotlib.pyplot as plt
plt.imshow(A.real)

# 2 - fill it!

There is also a [valuable comment in the video](https://www.youtube.com/watch?v=d3f3JRo0WUo), that we prefer a dense A. But we will ignore this for the lab and the homework. Just keep this in mind :)

Formulas from here: https://arxiv.org/pdf/2203.10236.pdf.

$\begin{bmatrix}
    A & -\sqrt{I - A^\dagger A} \\
    +\sqrt{I - A^\dagger A} & A
\end{bmatrix}$

or

$\begin{bmatrix}
    A & \sqrt{I - A^\dagger A} \\
    \sqrt{I - A^\dagger A} & -A
\end{bmatrix}$

We will follow the second formula.

NB: But for our task I observed, that lower corner is better be filled with $-A^\dagger$ to satisfy a better approximation. This may be a technical aspect for a particular problem of a sparse matrices.

ALSO: this tutorial (https://pennylane.ai/qml/demos/tutorial_intro_qsvt/) supports my finding, and fixing the matrix to be different. Note, that roots are different as well.

$\begin{bmatrix}
    A & \sqrt{I - AA^\dagger } \\
    \sqrt{I - A^\dagger A} & -A^\dagger
\end{bmatrix}$

In [32]:
# reserve a place for an ancillary garbage qubit
U = np.zeros(
    (A.shape[0] * 2, A.shape[1] * 2), 
    dtype=np.complex128
)

### Values of A and U should not exceed 1!

Here I may address you to the detailed comment to the homework #2 of 2023.

In [ ]:
A /= 4  # or take max(of a matrix) into account
print(max(A.flatten()))

### Put A in its places:

In [ ]:
U[:A.shape[0], :A.shape[0]] = A
U[-A.shape[0]:, -A.shape[0]:] = -A.conj().T
plt.imshow(U.real)

### Compute and position the root

In [ ]:
from scipy.linalg import sqrtm


root = ...
root2 = 

U[:A.shape[0], -A.shape[0]:] = root
U[-A.shape[0]:, :A.shape[0]] = root2

plt.imshow(U.real)

In [ ]:
# (U @ U.conj().T).round(3)

In [ ]:
u, s, vd = np.linalg.svd(U)

# exclude singular numbers and multiply 2 unitaries
U_ = u @ vd

plt.imshow(U_.real)

# 3 - simulate wisely!

In [ ]:
from qiskit import QuantumCircuit, execute, transpile, BasicAer

qc = QuantumCircuit(round(math.log2(U.shape[0])))

# let us init the state of |1> + |6>
# we expect the result:    |2> + |6>

# |1> + |6>  = |001> + |110> = (IxIxX)(GHZ)

# GHZ
...

# X
qc.x(0)

# PAY ATTENTION TO ANCILLARY VALUES! How many acillas should be use?
qc.unitary(U_, range(4))

qc.measure_all()

In [ ]:
counts = execute(
    qc, 
    BasicAer.get_backend('qasm_simulator'),
    shots=1000000).result().get_counts()
counts

In [ ]:
# what is the postprocessing step?
clean_counts = ... 

In [ ]:
clean_counts
from qiskit.visualization import plot_histogram
plot_histogram(clean_counts)

In [ ]:
qct = transpile(
    qc, BasicAer.get_backend("qasm_simulator"), 
    basis_gates=["rxx", "rx", "rz"],
    # basis_gates=["cz", "rx", "rz"],
    # basis_gates=["cx", "rx", "rz"],
)

In [ ]:
# qct.draw('mpl')
# qct.depth()
# qct.count_ops()

## Beyond that

Block encoding is not the best and not the only way to put a matrix inside unitary. Current SOTA is called QSVT. It is referred in the [abovementioned tutorial](https://pennylane.ai/qml/demos/tutorial_intro_qsvt/).